Visualize images generated by ImageDataGenerator

In [ ]:
import glob
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

In [ ]:
train_csv_path = '../input/cassava-leaf-disease-classification/train.csv'
json_path = '../input/cassava-leaf-disease-classification/label_num_to_disease_map.json'
img_dir = '../input/cassava-leaf-disease-classification/train_images/'

Load class name and id of the dataset

In [ ]:
with open (json_path, 'r') as file:
    class_names = json.load(file)

#print id and class_names 
class_names

Read all data from csv file

In [ ]:
train_data = pd.read_csv(train_csv_path)
train_data.head()

General data analysis, total number of images and number of images class-wise

In [ ]:
lc = train_data.label.value_counts()
class_count={}
for i in range(len(lc)):
  class_count[i] = lc[i]
print(class_count)

In [ ]:
print('total images-', train_data.shape[0])
plt.bar(class_count.keys(),class_count.values())
plt.xlabel('class id')
plt.ylabel('number of images')

Change label data type from int to str for ImageGeneretor

In [ ]:
train_data.label = train_data.label.astype('str')
print(train_data.dtypes)

Define funtion to visualize random / class-wise images

In [ ]:
def visualize_images(data,row=1,col=3):
    fig = plt.figure(figsize=(10,10))
#     row, col = 3,3
    for i in range(row*col):
        fig.add_subplot(row, col, i+1).set_title(class_names[data.label[i]])
        img = load_img(img_dir + data.image_id[i])
        img = img_to_array(img, dtype = np.uint8)
        plt.imshow(img)
    plt.show()

Select few random/class-wise data for visualization

In [ ]:
random_data = train_data.sample(10).reset_index(drop=True)
# class_0 = train_data[train_data.label == "0"].sample(10).reset_index(drop=True)

Visualize images

In [ ]:
visualize_images(random_data,1,3)
# visualize_images(class_0,1,3)

In [ ]:
IMG_SIZE = 512
batch_size = 64

Deine train and validation image generator using flow_from_dataframe. Apply augmentation accordingly, 

In [ ]:
train_generator = ImageDataGenerator(
                                    rotation_range=90,
#                                     width_shift_range=0.2,
#                                     height_shift_range=0.2,
#                                     brightness_range=[0.1,0.9],
                                    shear_range=25,
#                                     zoom_range=0.3,
#                                     channel_shift_range=0.2,
#                                     horizontal_flip=True,
#                                     vertical_flip=True,
                                    validation_split=0.1,
) \
        .flow_from_dataframe(
                            dataframe = train_data,
                            directory = img_dir,
                            x_col = "image_id",
                            y_col = "label",
                            target_size = (IMG_SIZE, IMG_SIZE),
                            class_mode = "categorical",
                            batch_size = batch_size,
                            shuffle = True,
                            subset = "training")

In [ ]:
val_generator = ImageDataGenerator(
                                    validation_split = 0.1
) \
        .flow_from_dataframe(
                            dataframe= train_data,
                            directory = img_dir,
                            x_col = "image_id",
                            y_col = "label",
                            target_size = (IMG_SIZE, IMG_SIZE),
                            class_mode = "categorical",
                            batch_size = batch_size,
                            shuffle = True,
                            subset = "validation")

load images from validation generator to check

In [ ]:
img_, label_ = val_generator.next()
print(img_.shape, label_.shape)

Define funtion to visualize generated images with label

In [ ]:
def visualize_images_from_datagen(data, label_data,row=1,col=3):
    fig = plt.figure(figsize=(10,10))
#     row, col = 3,3
    for i in range(row*col):
#         print(label_data[i])
        label = int(np.argmax([label_data[i]], axis =1))
#         print(label)
        fig.add_subplot(row, col, i+1).set_title(class_names[str(label)])
        img = data[i]
        img = np.uint8(img)#.astpye(np.uint8)
        plt.imshow(img)
    plt.show()

Select some images for visualization

In [ ]:
img_data = img_[0:10]
label_data =label_[0:10]
# img_data.shape
visualize_images_from_datagen(img_data, label_data, row=3,col=3)

Visualize Augmented images and their labels from train generator

In [ ]:
img_, label_ = train_generator.next()
print(img_.shape, label_.shape)
img_data = img_[0:10]
label_data =label_[0:10]
# img_data.shape
visualize_images_from_datagen(img_data, label_data, row=3,col=3)

In [ ]:
# for i, img in enumerate(img_):
#     plt.imshow(np.uint8(img))
#     plt.show()
#     if(i==15):
#         break